In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
#SETTING UP SELENIUM WEBDRIVER TO BE A HEADLESS BROWSER IN ORDER TO BYPASS CAPTCHAS AND WEB-BLOCKING
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"  # complete

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")


In [5]:
driver = uc.Chrome(options=options)

In [6]:
#THESE SETTING UP URL'S AND INITIALIZING SEARCH VARIABLES
import urllib.parse
business_type = 'landscape'
business_city = urllib.parse.quote('Baton Rouge')
business_state = urllib.parse.quote('LA')

In [7]:
####GATHERING YELLOW PAGES LEADS####

In [8]:
yellow_pages_url = f'https://www.yellowpages.com/search?search_terms={business_type}&geo_location_terms={business_city}%2C+{business_state}'
yp_page = requests.get(yellow_pages_url)
yp_soup = BeautifulSoup(yp_page.text,'html')

In [9]:
companies = yp_soup.find_all(class_='result')
#print(companies)

In [10]:
yp_business_names = []
yp_phones = []
yp_address_list = []

In [11]:
for company in yp_soup.find_all('div',class_='result'):
    name = company.find('a',class_='business-name')
    b_name = name.text.strip() if name else 'None'
    #print(b_name)
    phone = company.find('div',class_='phones phone primary')
    phone_num = phone.text.strip() if phone else 'None'
    #print(phone_num)
    address = company.find('div',class_='street-address')
    street_address = address.text.strip() if address else 'None'
    #print(street_address)
    yp_business_names.append(b_name)
    yp_phones.append(phone_num)
    yp_address_list.append(street_address)

In [12]:
df = pd.DataFrame({
    'Company Name': yp_business_names,
    'Phone Number': yp_phones,
    'Address': yp_address_list,
})
df = df.sort_values(by='Company Name')
display(df.style.hide_index())

/var/folders/zw/m6lz6_8d3jl6q1hv5qg_dm040000gn/T/ipykernel_46486/2299381379.py:7: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  display(df.style.hide_index())


Company Name,Phone Number,Address
A Green Affair,(225) 773-3152,None
ADAMS HANDYMAN SERVICES,(888) 316-5235,"PO Box,"
Ascension Tree And Stump LLC,(888) 679-1615,None
Baton Rouge Horticultural Service,(225) 924-5884,2222 Bardwell Dr
Blue Skies Landscape LLC,(225) 772-4183,None
Bonfiglio Lawn & Landscape,(225) 447-8880,7195 Scobell Dr
C & M Outdoor Power Equipment,(225) 271-5352,8472 Florida Blvd
Chop Chop Landscaping in Baton Rouge,(225) 240-8586,301 Main St
Evergreen landscape,(225) 803-5979,11675 E Black Oak Dr
"HD Lawn and Landscape, LLC",(225) 402-7978,13931 Rampart Ct


In [13]:
####GATHER YELP LEADS####

In [14]:
yelp_url = f'https://www.yelp.com/search?find_desc={business_type}&find_loc={business_city},{business_state}'
yelp_page = requests.get(yelp_url)
yelp_soup = BeautifulSoup(yelp_page.text,'html')
print(yelp_soup)

<html><head><title>yelp.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMARNWXMbmacdAAYV46HA==','hsh':'3BD2468BAE4D73BEA0B5DE8314D745','t':'bv','s':46977,'e':'8456f6f8abb821760db20f4e81a8b8a24e38c3cc766f7d727944012d6a6c75fc','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>
